# Análisis Exploratorio de Datos (EDA) - Tienda Online

Este notebook presenta un análisis exploratorio de ventas de una tienda online utilizando el dataset **Online Retail II**.
El objetivo es comprender el comportamiento de los clientes, patrones de ventas, productos destacados y oportunidades para optimizar las decisiones del negocio.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración de estilo gráfico
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.titlesize"] = 16
plt.rcParams["axes.labelsize"] = 13


## Carga y combinación de datos

Se cargan las dos hojas del archivo Excel correspondientes a los años 2009-2010 y 2010-2011, y luego se concatenan.


In [ ]:
# Cargar el archivo Excel
file_path = "online_retail_II.xlsx"
xls = pd.ExcelFile(file_path)

# Leer hojas
df1 = xls.parse('Year 2009-2010')
df2 = xls.parse('Year 2010-2011')

# Agregar columna de año para identificar el origen
df1['Year'] = '2009-2010'
df2['Year'] = '2010-2011'

# Concatenar ambas hojas
df = pd.concat([df1, df2], ignore_index=True)
df.head()

## Preprocesamiento de los datos

Se realiza limpieza y enriquecimiento del dataset:
- Eliminación de nulos irreparables.
- Creación de columnas útiles.
- Identificación de devoluciones.


In [ ]:
# Eliminar filas sin Customer ID o Description
df.dropna(subset=['Customer ID', 'Description'], inplace=True)

# Limpiar y normalizar descripciones
df['Description'] = df['Description'].str.strip().str.lower()

# Convertir tipos y crear columnas útiles
df['Customer ID'] = df['Customer ID'].astype(int)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['Price']
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceDay'] = df['InvoiceDate'].dt.day
df['InvoiceWeekday'] = df['InvoiceDate'].dt.day_name()
df['IsReturn'] = df['Invoice'].astype(str).str.startswith('C') | (df['Quantity'] < 0)

# Filtrar precios positivos
df = df[df['Price'] > 0]


## Estadísticas descriptivas generales

In [ ]:
# Estadísticas para variables numéricas
df[['Quantity', 'Price', 'TotalPrice']].describe()

## Estadísticas descriptivas por país

In [ ]:
# Promedios por país
grouped_country = df.groupby('Country')[['Quantity', 'Price', 'TotalPrice']].agg(['mean', 'median', 'std'])
grouped_country.head(10)

## Distribución de precios

In [ ]:
sns.histplot(df['Price'], bins=100, kde=True)
plt.xlim(0, 50)
plt.title('Distribución de Precios')
plt.xlabel('Precio (€)')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

## Distribución de TotalPrice por orden

In [ ]:
sns.histplot(df['TotalPrice'], bins=100, kde=True)
plt.xlim(0, 500)
plt.title('Distribución de TotalPrice')
plt.xlabel('TotalPrice por línea de factura')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

## Detección de outliers

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

sns.boxplot(y=df['Quantity'], ax=axs[0])
axs[0].set_title('Boxplot de Cantidad')
axs[0].set_ylim(-100, 500)

sns.boxplot(y=df['Price'], ax=axs[1])
axs[1].set_title('Boxplot de Precio')
axs[1].set_ylim(0, 100)

sns.boxplot(y=df['TotalPrice'], ax=axs[2])
axs[2].set_title('Boxplot de TotalPrice')
axs[2].set_ylim(0, 1000)

plt.tight_layout()
plt.show()

## Correlación entre variables

In [ ]:
sns.heatmap(df[['Quantity', 'Price', 'TotalPrice']].corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

## Gasto medio por país

In [ ]:
avg_price_per_country = df.groupby('Country')['TotalPrice'].mean().sort_values(ascending=False).head(10)
sns.barplot(x=avg_price_per_country.values, y=avg_price_per_country.index)
plt.title('Gasto Medio por Orden - Top 10 Países')
plt.xlabel('Promedio de TotalPrice (€)')
plt.ylabel('País')
plt.tight_layout()
plt.show()

## Productos más vendidos

In [ ]:
top_products = df[~df['IsReturn']].groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=top_products.values, y=top_products.index)
plt.title('Top 10 Productos Más Vendidos')
plt.xlabel('Cantidad Vendida')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

## Productos que generan más ingresos

In [ ]:
top_revenue = df[~df['IsReturn']].groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=top_revenue.values, y=top_revenue.index)
plt.title('Top 10 Productos por Ingreso Total')
plt.xlabel('Ingresos (€)')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

## Productos más devueltos

In [ ]:
top_returns = df[df['IsReturn']].groupby('Description')['Quantity'].sum().abs().sort_values(ascending=False).head(10)
sns.barplot(x=top_returns.values, y=top_returns.index)
plt.title('Top 10 Productos Más Devueltos')
plt.xlabel('Cantidad Devuelta')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

## Ingresos por país

In [ ]:
revenue_country = df[~df['IsReturn']].groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=revenue_country.values, y=revenue_country.index)
plt.title('Top 10 Países por Ingresos Generados')
plt.xlabel('Ingresos (€)')
plt.ylabel('País')
plt.tight_layout()
plt.show()

## Porcentaje de devoluciones por país

In [ ]:
returns_ratio = df.groupby('Country')['IsReturn'].mean().sort_values(ascending=False).head(10) * 100
sns.barplot(x=returns_ratio.values, y=returns_ratio.index)
plt.title('Top 10 Países con Mayor Porcentaje de Devoluciones')
plt.xlabel('% Devoluciones')
plt.ylabel('País')
plt.tight_layout()
plt.show()

## Ventas por día de la semana

In [ ]:
weekday_sales = df[~df['IsReturn']].groupby('InvoiceWeekday')['TotalPrice'].sum().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
sns.barplot(x=weekday_sales.index, y=weekday_sales.values)
plt.title('Ingresos por Día de la Semana')
plt.xlabel('Día')
plt.ylabel('Ingresos (€)')
plt.tight_layout()
plt.show()